In [33]:
import requests
import csv
import re
import json
from collections import defaultdict
from threading import Thread
import pandas as pd
import logging
from datetime import datetime
from os import path
import os
from queue import Queue
import time

In [3]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("uptime")
logger.setLevel(logging.INFO)
filename = "./logs/uptime_{}.log".format(datetime.utcnow().strftime('%Y_%m_%d_%H:%M:%S'))
file_handler = logging.FileHandler(filename)
file_handler.setLevel(logging.INFO)
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)
logger.addHandler(file_handler)
logger.addHandler(console_handler)

In [10]:
def get_information(url, method, params) -> dict:
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    if r.status_code != 200:
        print("Error: Return status code %s" % r.status_code)
        exit(1)
    content = json.loads(r.content)
    return content['result']

In [180]:
def getCommittees(shard, epoch) -> dict:
    url = endpoint[shard]
    method = "hmy_getValidators"
    params = [epoch]
    return get_information(url, method, params)


In [5]:
def getBlockSigner(shard, block):
    url = endpoint[shard]
    payload = json.dumps({
        "jsonrpc": "2.0",
        "method": "hmy_getBlockSigners",
        "params": [
            hex(block)
        ],
        "id": 1
    })
    headers = {
        'Content-Type': 'application/json'
    }
    response = requests.request('POST', url, headers=headers, data=payload, allow_redirects=False, timeout=30)
    try:
        returned = json.loads(response.content)["result"]
        return returned
    except Exception:  # Catch all to not halt
        t= {
            'block-num': block,
            'reason': f"Failed to json load block {block}. Content: {response.content}"
        }
        logger.info(json.dumps(t))
        print(f"\n[!!!] Failed to json load block {block}. Content: {response.content}\n")

In [167]:
def getLatestHeader(shard) -> dict:
    url = endpoint[shard]
    method = 'hmy_latestHeader'
    params = []
    return get_information(url, method, params)

In [8]:
def getBlockByNumber(shard, number) -> dict:
    url = endpoint[shard]
    method = 'hmyv2_getBlockByNumber'
    params = [number, {"fullTx":True,"inclTx":True,"InclStaking":True}]
    return get_information(url, method, params)

In [11]:
def getBlockByNumber(shard, number) -> dict:
    url = endpoint[shard]
    method = 'hmy_getBlockByNumber'
    params = [hex(number), False]
    return get_information(url, method, params)

In [9]:
def IsFirstBlockOfEpoch(block) -> bool:
    if 1 < block < mainnetEpochBlock1:
        return False
    elif block == 1:
        return True
    else:
        return ((block - mainnetEpochBlock1) % blocksPerEpoch == 0)

In [162]:
def getSigningAddress(shard, epoch) -> list:
    for i in getCommittees(shard, epoch)['validators']:
        if i['address'] != 'one1qqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqquzw7vz':
            address.append(i['address'])
    return address

In [11]:
def getSigningPerc(sign, count):
    perc = dict()
    for k, v in count.items():
        if k in sign:
            perc[k] = sign[k]/v
        else:
            perc[k] = 0
    return perc

In [12]:
def get_information(url, method, params) -> dict:
    headers = {'Content-Type': 'application/json; charset=utf8'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    try:
        r = requests.post(url, headers=headers, data = json.dumps(data))
    except requests.exceptions.ConnectionError as e:
        print("Error: connection error")
        time.sleep(5)
        return None
    if r.status_code != 200:
        print("Error: Return status code %s" % r.status_code)
        return None
    try:
        r.json()
    except ValueError:
        print("Error: Unable to read JSON reply")
        return None
    return r.json()

In [29]:
import pickle
file = "../pickle/committee_count_mainnet.pkl"
with open(file, 'rb') as f:
    count = pickle.load(f)

In [39]:
import pickle
file = "../pickle/signer_count_mainnet.pickle"
with open(file, 'rb') as f:
    count = pickle.load(f)

In [24]:
file = "../pickle/shard_info_mainnet.pickle"
with open(file, 'rb') as f:
    info = pickle.load(f)

In [ ]:
mainnetEpochBlock1 = 344064
blocksPerEpoch = 16384
endpoint = ['https://api.s0.t.hmny.io/', 'https://api.s1.t.hmny.io/', 'https://api.s2.t.hmny.io/', 'https://api.s3.t.hmny.io/']

num = []
for j in range(4):
    num.append(getLatestHeader(j)['blockNumber'])

In [ ]:
if __name__ == "__main__":
    sign = defaultdict(int)
    count = defaultdict(int)
    def collect_data(shard):
        global sign, count
        epoch = 0
        for i in range(1,num[shard]+1):
            if IsFirstBlockOfEpoch(i):
                address = getSigningAddress(shard, epoch)
                epoch += 1
            signer = getBlockSigner(shard, block)
            for j in address:
                if j in signer:
                    sign[j] += 1
                count[j] += 1
            if (shard == 0) and (i % 1000 == 0):
                res = getBlockByNumber(shard, i)
                time = datetime.fromtimestamp(res['timestamp']).strftime('%Y_%m_%d_%H:%M:%S')
                perc = getSigningPerc(sign, count)
                t = {
                     "timestamp": time,
                     "signing-percentage": perc,
                    }
                logger.info(json.dumps(t))
        if shard == 0:
            res = getBlockByNumber(shard, i)
            time = datetime.fromtimestamp(res['timestamp']).strftime('%Y_%m_%d_%H:%M:%S')
            perc = getSigningPerc(sign, count)
            t = {
                 "timestamp": time,
                 "signing-percentage": perc,
                }
            logger.info(json.dumps(t))
    
    threads = []
    for x in range(len(endpoint)):
        threads.append(Thread(target = collect_data, args = [x]))
    for t in threads:
        t.start()
    for t in threads:
        t.join()
        
    perc = getSigningPerc(sign, count)
    df = pd.DataFrame(perc.items(), columns=['address', 'signing-percentage'])
    df.to_csv("block_signer.csv")
    print("Successfully saving signing percentage to csv")

In [14]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
from os import path

base = path.dirname(path.realpath("block_signers.ipynb"))
json_dir = path.abspath(path.join(base, 'credential'))
log_dir = path.abspath(path.join(base, 'logs'))

# Fetch the service account key JSON file contents
cred = credentials.Certificate(path.join(json_dir, "harmony-explorer-mainnet-firebase-adminsdk.json"))

# Initialize the app with a service account, granting admin privileges
firebase_admin.initialize_app(cred, {'databaseURL': "https://harmony-explorer-mainnet.firebaseio.com"})
db = firestore.client()


In [97]:
block_ref.set(signer_dict)

update_time {
  seconds: 1589476945
  nanos: 726860000
}

In [ ]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db


base = path.dirname(path.realpath("block_signers.ipynb"))
json_dir = path.abspath(path.join(base, 'credential'))
log_dir = path.abspath(path.join(base, 'logs'))

# Fetch the service account key JSON file contents
cred = credentials.Certificate(path.join(json_dir, "harmony-explorer-mainnet-firebase-adminsdk.json"))


# Initialize the app with a service account, granting admin privileges
firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://harmony-explorer-mainnet.firebaseio.com/'
})

# As an admin, the app has access to read and write all data, regradless of Security Rules
ref = db.reference('block-signer')
shard_ref = ref.child('shard0')
block_ref = shard_ref.child("1")
block_ref.set({
    'signers':signer
})

In [ ]:
ref = db.reference('block-signer')
shard_ref = ref.child('shard0')
block_ref = shard_ref.child("1")
block_ref.set(
    signer_dict
)